## Importing Libraries

In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import neighbors
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder

import getpass 
password = getpass.getpass()

## Data Exploration + Cleaning

In [237]:
# Import the 'film', 'rental' 'category', 'inventory, and 'payments' tables from the sakila database

connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
query = '''SELECT * FROM sakila.film f
JOIN sakila.inventory i USING (film_id)
JOIN sakila.rental fa USING (inventory_id)
JOIN sakila.film_category fc USING (film_id)
JOIN sakila.category c USING (category_id)
JOIN sakila.payment p USING (rental_id);'''

pd.set_option('display.max_columns', None) # Used to show all columns

data_sakila = pd.read_sql_query(query, engine)
data_sakila

,rental_id,category_id,film_id,inventory_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update,store_id,last_update,rental_date,customer_id,return_date,staff_id,last_update,last_update,name,last_update,payment_id,customer_id,staff_id,amount,payment_date,last_update
0,10895,1,19,93,AMADEUS HOLY,A Emotional Display of a Pioneer And a Technic...,2006,1,None,6,0.99,113,20.99,PG,"Commentaries,Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,1,2006-02-15 05:09:17,2005-08-02 01:16:59,77,2005-08-03 02:41:59,2,2006-02-15 21:30:53,2006-02-15 05:07:09,Action,2006-02-15 04:46:27,2104,77,1,0.99,2005-08-02 01:16:59,2006-02-15 22:12:50
1,12268,1,19,93,AMADEUS HOLY,A Emotional Display of a Pioneer And a Technic...,2006,1,None,6,0.99,113,20.99,PG,"Commentaries,Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,1,2006-02-15 05:09:17,2005-08-18 04:26:54,39,2005-08-23 06:40:54,2,2006-02-15 21:30:53,2006-02-15 05:07:09,Action,2006-02-15 04:46:27,1093,39,2,0.99,2005-08-18 04:26:54,2006-02-15 22:12:38
2,3150,1,19,94,AMADEUS HOLY,A Emotional Display of a Pioneer And a Technic...,2006,1,None,6,0.99,113,20.99,PG,"Commentaries,Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,1,2006-02-15 05:09:17,2005-06-20 20:35:28,34,2005-06-26 01:01:28,1,2006-02-15 21:30:53,2006-02-15 05:07:09,Action,2006-02-15 04:46:27,930,34,1,0.99,2005-06-20 20:35:28,2006-02-15 22:12:36
3,5072,1,19,94,AMADEUS HOLY,A Emotional Display of a Pioneer And a Technic...,2006,1,None,6,0.99,113,20.99,PG,"Commentaries,Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,1,2006-02-15 05:09:17,2005-07-09 05:01:58,254,2005-07-18 08:17:58,2,2006-02-15 21:30:53,2006-02-15 05:07:09,Action,2006-02-15 04:46:27,6848,254,1,3.99,2005-07-09 05:01:58,2006-02-15 22:14:52
4,9080,1,19,94,AMADEUS HOLY,A Emotional Display of a Pioneer And a Technic...,2006,1,None,6,0.99,113,20.99,PG,"Commentaries,Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,1,2006-02-15 05:09:17,2005-07-30 08:02:39,276,2005-08-06 12:02:39,1,2006-02-15 21:30:53,2006-02-15 05:07:09,Action,2006-02-15 04:46:27,7479,276,1,1.99,2005-07-30 08:02:39,2006-02-15 22:15:16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16039,962,16,989,4535,WORKING MICROCOSMOS,A Stunning Epistle of a Dentist And a Dog who ...,2006,1,None,4,4.99,74,22.99,R,"Commentaries,Deleted Scenes",2006-02-15 05:03:42,2,2006-02-15 05:09:17,2005-05-30 18:45:17,520,2005-06-05 22:47:17,1,2006-02-15 21:30:53,2006-02-15 05:07:09,Travel,2006-02-15 04:46:27,13988,520,1,6.99,2005-05-30 18:45:17,2006-02-15 22:20:53
16040,1292,16,989,4535,WORKING MICROCOSMOS,A Stunning Epistle of a Dentist And a Dog who ...,2006,1,None,4,4.99,74,22.99,R,"Commentaries,Deleted Scenes",2006-02-15 05:03:42,2,2006-02-15 05:09:17,2005-06-15 09:03:52,178,2005-06-21 07:53:52,1,2006-02-15 21:30:53,2006-02-15 05:07:09,Travel,2006-02-15 04:46:27,4823,178,1,6.99,2005-06-15 09:03:52,2006-02-15 22:13:47
16041,4108,16,989,4535,WORKING MICROCOSMOS,A Stunning Epistle of a Dentist And a Dog who ...,2006,1,None,4,4.99,74,22.99,R,"Commentaries,Deleted Scenes",2006-02-15 05:03:42,2,2006-02-15 05:09:17,2005-07-07 06:38:31,66,2005-07-08 10:44:31,1,2006-02-15 21:30:53,2006-02-15 05:07:09,Travel,2006-02-15 04:46:27,1800,66,1,4.99,2005-07-07 06:38:31,2006-02-15 22:12:46
16042,8871,16,989,4535,WORKING MICROCOSMOS,A Stunning Epistle of a Dentist And a Dog who ...,2006,1,None,4,4.99,74,22.99,R,"Commentaries,Deleted Scenes",2006-02-15 05:03:42,2,2006-02-15 05:09:17,2005-07-30 00:12:41,382,2005-08-08 03:53:41,1,2006-02-15 21:30:53,2006-02-15 05:07:09,Travel,2006-02-15 04:46:27,10355,382,1,9.99,2005-07-30 00:12:41,2006-02-15 22:17:24


In [238]:
data_sakila.describe().T

,count,mean,std,min,25%,50%,75%,max
rental_id,16044.0,8025.371478,4632.777249,1.00,4013.75,8025.50,12037.25,16049.00
category_id,16044.0,8.363999,4.650700,1.00,4.00,8.00,13.00,16.00
film_id,16044.0,501.108888,288.513529,1.00,255.00,496.00,753.00,1000.00
inventory_id,16044.0,2291.842558,1322.210643,1.00,1154.00,2291.00,3433.00,4581.00
release_year,16044.0,2006.000000,0.000000,2006.00,2006.00,2006.00,2006.00,2006.00
language_id,16044.0,1.000000,0.000000,1.00,1.00,1.00,1.00,1.00
rental_duration,16044.0,4.935490,1.401690,3.00,4.00,5.00,6.00,7.00
rental_rate,16044.0,2.942630,1.649678,0.99,0.99,2.99,4.99,4.99
length,16044.0,114.971080,40.102347,46.00,81.00,114.00,148.00,185.00
replacement_cost,16044.0,20.215443,6.081771,9.99,14.99,20.99,25.99,29.99


In [ ]:
# Check frequency of NaNs by column and determine how to handle them. 
# Looks like 'original_language_id' is mostly nan (and unhelpful) so we will drop the column later

data_sakila.isna().sum()

In [ ]:
# Look at and drop duplicates rows. This gives me an error message which I assume means there are no duplicates 
# (and may be a function of joining the tables via pandas instead of SQL)

print(data_sakila[data_sakila.duplicated()])
data_sakila.drop_duplicates()

In [ ]:
# Checking values in each column to see if they can be disposed of. This would make more sense as a loop or a display all...I know

data_sakila['language_id'].unique
data_sakila['release_year'].unique
data_sakila['rental_duration'].unique
data_sakila['rating'].unique
data_sakila['replacement_cost'].unique
data_sakila['special_features'].unique()

In [239]:
# Remove duplicate columns and those columns that will not work / that we do not want in the model

data = data_sakila.drop(['category_id', 'rental_id', 'inventory_id', 'description', 'release_year', 'language_id', 'original_language_id', 'last_update',
                     'last_update', 'rental_date', 'customer_id', 'return_date', 'staff_id', 'replacement_cost', 'payment_id', 'payment_date', 'special_features', 'title'], axis=1)

print(data.shape)
data.head()

(16044, 8)


,film_id,rental_duration,rental_rate,length,rating,store_id,name,amount
0,19,6,0.99,113,PG,1,Action,0.99
1,19,6,0.99,113,PG,1,Action,0.99
2,19,6,0.99,113,PG,1,Action,0.99
3,19,6,0.99,113,PG,1,Action,3.99
4,19,6,0.99,113,PG,1,Action,1.99


In [240]:
# Create a new column for duration of customer rental which is return date - rental date (a timedelta64 dtype):

data['duration_of_rental'] = (data_sakila['return_date'] - data_sakila['rental_date'])

data.head() # This returns 184 NaTs in the new column

sum(data['duration_of_rental'].isna())

# Replace the NaTs with the column median 

time_median = data['duration_of_rental'].median()

time_median

data['duration_of_rental'] = data['duration_of_rental'].replace({np.nan: time_median}).dt.days

data.head()

/var/folders/kp/1thpb47s3s540gy9cm49qr740000gn/T/ipykernel_14646/644870431.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['duration_of_rental'] = (data_sakila['return_date'] - data_sakila['rental_date'])
/var/folders/kp/1thpb47s3s540gy9cm49qr740000gn/T/ipykernel_14646/644870431.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['duration_of_rental'] = data['duration_of_rental'].replace({np.nan: time_median}).dt.days


,film_id,rental_duration,rental_rate,length,rating,store_id,name,amount,duration_of_rental
0,19,6,0.99,113,PG,1,Action,0.99,1
1,19,6,0.99,113,PG,1,Action,0.99,5
2,19,6,0.99,113,PG,1,Action,0.99,5
3,19,6,0.99,113,PG,1,Action,3.99,9
4,19,6,0.99,113,PG,1,Action,1.99,7


In [241]:
# Create the target - likelihood that the film is rented in (May)

data['may_target'] = pd.DatetimeIndex(data_sakila['rental_date']).month

data.head()

# Then convert that into a boolean operator so if May (5) then True. Everything else is False. 

def boolean(x):
    if x==5:
        return 1
    else:
        return 0

data['may_target'] = list(map(boolean,data['may_target']))

print(data.shape)
data.head(10)

(16044, 10)


/var/folders/kp/1thpb47s3s540gy9cm49qr740000gn/T/ipykernel_14646/1092396033.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['may_target'] = pd.DatetimeIndex(data_sakila['rental_date']).month
/var/folders/kp/1thpb47s3s540gy9cm49qr740000gn/T/ipykernel_14646/1092396033.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['may_target'] = list(map(boolean,data['may_target']))


,film_id,rental_duration,rental_rate,length,rating,store_id,name,amount,duration_of_rental,may_target
0,19,6,0.99,113,PG,1,Action,0.99,1,0
1,19,6,0.99,113,PG,1,Action,0.99,5,0
2,19,6,0.99,113,PG,1,Action,0.99,5,0
3,19,6,0.99,113,PG,1,Action,3.99,9,0
4,19,6,0.99,113,PG,1,Action,1.99,7,0
5,19,6,0.99,113,PG,1,Action,1.99,6,0
6,19,6,0.99,113,PG,1,Action,0.99,1,1
7,19,6,0.99,113,PG,1,Action,0.99,3,0
8,19,6,0.99,113,PG,1,Action,0.99,5,0
9,19,6,0.99,113,PG,1,Action,0.99,5,0


In [242]:
# Convert store_id to a categorical variable (either store one or store 2) before we scale/encode

def alpha(i):
    if i==1:
        return 'A'
    else:
        return 'B'

data['store_id'] = list(map(alpha,data['store_id']))

data.head(20)

/var/folders/kp/1thpb47s3s540gy9cm49qr740000gn/T/ipykernel_14646/1330393693.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['store_id'] = list(map(alpha,data['store_id']))


,film_id,rental_duration,rental_rate,length,rating,store_id,name,amount,duration_of_rental,may_target
0,19,6,0.99,113,PG,A,Action,0.99,1,0
1,19,6,0.99,113,PG,A,Action,0.99,5,0
2,19,6,0.99,113,PG,A,Action,0.99,5,0
3,19,6,0.99,113,PG,A,Action,3.99,9,0
4,19,6,0.99,113,PG,A,Action,1.99,7,0
5,19,6,0.99,113,PG,A,Action,1.99,6,0
6,19,6,0.99,113,PG,A,Action,0.99,1,1
7,19,6,0.99,113,PG,A,Action,0.99,3,0
8,19,6,0.99,113,PG,A,Action,0.99,5,0
9,19,6,0.99,113,PG,A,Action,0.99,5,0


In [ ]:
# corr_matrix=data.corr(method='pearson')
# fig, ax = plt.subplots(figsize=(10, 8))
# ax = sns.heatmap(corr_matrix, cmap='mako_r', annot=True)
# plt.show()

# Correlation matrix looks good. Low multicolinearity between the independent variables (features). Should remove categoricals but too lazy.

## Making the Model

In [252]:
# Split the data into features and target

y = data['may_target']
X = data.drop(['may_target', 'film_id', 'title'], axis=1)

# Train transformation on training set only

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)

In [253]:
# Isolate the numerical features we want (and remove film_id to be concatenated back at the end):

X_train_num = X_train.select_dtypes(include = np.number)

# Use minmaxscaler to normalize the numerical features:

transformer = MinMaxScaler().fit(X_train_num) 
X_train_normalized = transformer.transform(X_train_num)
X_train_norm = pd.DataFrame(X_train_normalized, columns=X_train_num.columns)

X_train_norm # Check that we dropped categoricals and values are between 0 and 1

,rental_duration,rental_rate,length,amount,duration_of_rental
0,1.00,0.0,0.064748,0.082569,0.333333
1,0.50,1.0,0.244604,0.416180,0.555556
2,1.00,0.0,0.460432,0.165972,0.888889
3,0.00,0.0,0.172662,0.249374,0.444444
4,1.00,0.5,0.237410,0.249374,0.555556
...,...,...,...,...,...
12830,0.00,0.5,0.539568,0.416180,0.444444
12831,1.00,1.0,0.856115,0.499583,0.777778
12832,0.25,0.5,0.697842,0.249374,0.000000
12833,0.75,0.5,0.964029,0.249374,0.222222


In [255]:
# Encode the categorical variables (only the 'ratings' and 'category_name' columns in this case)

X_train_categorical = X_train.select_dtypes(include = object)
X_train_categorical 

encoder = OneHotEncoder(drop='first').fit(X_train_categorical)
encoded = encoder.transform(X_train_categorical).toarray()

cols = encoder.get_feature_names(input_features=X_train_categorical.columns)

X_train_cat = onehot_encoded = pd.DataFrame(encoded, columns=cols)
X_train_cat.head()

,rating_NC-17,rating_PG,rating_PG-13,rating_R,store_id_B,name_Animation,name_Children,name_Classics,name_Comedy,name_Documentary,name_Drama,name_Family,name_Foreign,name_Games,name_Horror,name_Music,name_New,name_Sci-Fi,name_Sports,name_Travel
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [256]:
X_train_transformed = np.concatenate([X_train_norm, X_train_cat], axis=1)
X_train_transformed

array([[1.        , 0.        , 0.0647482 , ..., 0.        , 0.        ,
        1.        ],
       [0.5       , 1.        , 0.24460432, ..., 0.        , 1.        ,
        0.        ],
       [1.        , 0.        , 0.46043165, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.25      , 0.5       , 0.69784173, ..., 0.        , 0.        ,
        0.        ],
       [0.75      , 0.5       , 0.96402878, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.5       , 0.53956835, ..., 0.        , 0.        ,
        0.        ]])

In [257]:
classification = LogisticRegression(random_state=0, solver='lbfgs',
                  multi_class='multinomial').fit(X_train_transformed, y_train)

In [259]:
# for numericals
X_test_num = X_test.select_dtypes(include = np.number)
X_test_num 

# Scaling data
# we use the transformer that was trained on the training data
X_test_normalized = transformer.transform(X_test_num)
X_test_norm = pd.DataFrame(X_test_normalized)

In [260]:
# for categoricals
X_test_categorical = X_test.select_dtypes(include = object)

# Encode again
encoder = OneHotEncoder(drop='first').fit(X_test_categorical)
encoded = encoder.transform(X_test_categorical).toarray()

cols = encoder.get_feature_names(input_features=X_test_categorical.columns)

X_test_cat = onehot_encoded = pd.DataFrame(encoded, columns=cols)
X_test_cat.head()

,rating_NC-17,rating_PG,rating_PG-13,rating_R,store_id_B,name_Animation,name_Children,name_Classics,name_Comedy,name_Documentary,name_Drama,name_Family,name_Foreign,name_Games,name_Horror,name_Music,name_New,name_Sci-Fi,name_Sports,name_Travel
0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [261]:
X_test_transformed = np.concatenate([X_test_norm, X_test_cat], axis=1)

X_test_transformed

array([[0.5       , 0.        , 0.4676259 , ..., 0.        , 0.        ,
        1.        ],
       [0.        , 0.        , 0.10791367, ..., 1.        , 0.        ,
        0.        ],
       [1.        , 0.5       , 0.53956835, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.25      , 0.        , 0.05755396, ..., 0.        , 0.        ,
        0.        ],
       [0.75      , 1.        , 0.07194245, ..., 0.        , 0.        ,
        0.        ],
       [0.5       , 0.        , 0.41007194, ..., 0.        , 0.        ,
        0.        ]])

In [262]:
predictions = classification.predict(X_test_transformed)
classification.score(X_test_transformed, y_test)

0.9267684636958554

In [266]:
print(y_test.value_counts())

0    2974
1     235
Name: may_target, dtype: int64


In [267]:
pd.Series(predictions).value_counts()

0    3209
dtype: int64

In [268]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[2974,    0],
       [ 235,    0]])

In [ ]:
# Not likely to have a lot of rentals in May. Most of the historical rentals are in August